### Computations with $S$-characters

The following inputs describe computations for the paper [Zeros of $S$-characters](https://arxiv.org/abs/2408.16785), in Julia sessions with 128 GB of memory.

Some of these computations take several hours of CPU time.

In [1]:
using S_characters

In [2]:
using Oscar

Run over all groups with at most 14 conjugacy classes.
(Currently there are no classifications of groups with more classes.)

This computation furnishes a proof of Theorem 1 in the [article mentioned above](https://arxiv.org/abs/2408.16785).

In [3]:
for k in 2:14
  for g in all_groups_with_class_number(k)
    t = character_table(g)
    res = s_characters(t, rational = false, ppow_nonzero = true)
    length(res) != 0 && println(describe(g), ": ", ItemQuantity(length(res), "solution"))
  end
end

A8: 1 solution


Run over all library character tables with at most 20 classes (currently 340 tables).

In [4]:
for nam in all_character_table_names(
             number_of_conjugacy_classes => 1:20, !is_duplicate_table)
  t = character_table(nam)
  res = s_characters(t, rational = false, ppow_nonzero = true)
  length(res) != 0 && println(nam, ": ", ItemQuantity(length(res), "solution"))
end

A8: 1 solution
A9: 3 solutions
J1: 1 solution
M12: 1 solution
U4(2): 2 solutions


Run over all Atlas library tables of order at most the order of A8.

This needs about 4000 seconds.

In [5]:
for nam in all_character_table_names(
             is_atlas_character_table, !is_duplicate_table,
             order => 2:div(factorial(8), 2),
             ordered_by = number_of_conjugacy_classes)
  t = character_table(nam)
  res = s_characters(t, rational = false, ppow_nonzero = true)
  length(res) != 0 && println(nam, ": ", ItemQuantity(length(res), "solution"))
end

A8: 1 solution


Run over all library character tables (*not nec.* Atlas tables) of perfect groups with at most 40 classes, excluding those tables for which Julia crashed (with 128GB of space). Search only for *rational* examples.

This needs about 30000 seconds.

In [6]:
crashes = [ "O8-(2)", "3D4(2)", "G2(4)", "He", "Ru", "ON" ];

for nam in all_character_table_names(
             is_perfect => true,
             !is_duplicate_table,
             number_of_conjugacy_classes => 2:40,
             ordered_by = number_of_conjugacy_classes)
  if !(nam in crashes)
    t = character_table(nam)
    res = s_characters(t, rational = true, ppow_nonzero = true)
    length(res) != 0 && println(nam, ": ", ItemQuantity(length(res), "solution"))
  end
end

A8: 1 solution
J1: 1 solution
M12: 1 solution
A9: 3 solutions
U4(2): 2 solutions
J2: 155 solutions
J3: 5 solutions
2F4(2)': 71 solutions
G2(3): 24 solutions
2.A8: 3 solutions
McL: 2588 solutions
HS: 93 solutions
A10: 1 solution
2^4.a8: 3 solutions
2^4:a8: 3 solutions
2.M12: 1 solution
M24: 223 solutions
L5(2): 11 solutions
S4(4): 133 solutions
L4(3): 6 solutions
S6(2): 1 solution
2.A9: 13 solutions
A11: 6 solutions
S4(5): 76 solutions
2.U4(2): 3 solutions
R(27): 9 solutions
2.J2: 2726 solutions
2.A10: 4 solutions

Run over all library character tables of non-perfect ATLAS groups with at most 40 classes, excluding those tables for which Julia crashed (with 128GB of space). Search only for *rational* examples.

This needs about 2000 seconds.

In [7]:
crashes2 = [ "HS.2", "G2(4).2", "McL.2" ];

for nam in all_character_table_names(
             is_atlas_character_table, !is_duplicate_table,
             is_perfect => false,
             number_of_conjugacy_classes => 2:40,
             ordered_by = number_of_conjugacy_classes)
  t = character_table(nam)
  if !(nam in crashes2)
    res = s_characters(t, rational = true, ppow_nonzero = true)
    length(res) != 0 && println(nam, ": ", ItemQuantity(length(res), "solution"))
  end
end

M12.2: 1 solution
J2.2: 12 solutions
2F4(2)'.2: 7 solutions
S4(4).2: 5 solutions
L5(2).2: 3 solutions
2.M12.2: 1 solution

Try some Atlas groups with more conjugacy classes, using the two-step approach. Search only for *rational* examples.

This needs about 8000 seconds.

In [8]:
res = s_characters_via_factor_group(character_table("2.A11"), # 49 classes
        [1, 2], rational = true, ppow_nonzero = true);
length(res)

59

In [9]:
res = s_characters_via_factor_group(character_table("2.HS"), # 42 classes
        [1, 2], rational = true, ppow_nonzero = true);
length(res)

2304

In [10]:
res = s_characters_via_factor_group(character_table("2^5.L5(2)"), # 41 classes
        [1, 2], rational = true, ppow_nonzero = true);
length(res)

2707

In [11]:
res = s_characters_via_factor_group(character_table("2^6:A8"), # 41 classes
        [1, 2, 3], rational = true, ppow_nonzero = true);
length(res)

9

Run over all groups of order less than 1536 and with at most 40 rational conjugacy classes. Search only for *rational* examples.

(This runs for a long time, due to the many groups, and without finding examples.)

In [ ]:
for n in 2:1535
  # p-groups cannot yield examples
  is_prime_power_with_data(n)[1] && continue
  for i in 1:number_of_small_groups(n)
    G = small_group(n, i)

    # omit abelian groups
    is_abelian(G) && continue

    # omit solvable groups with noncyclic centre or noncyclic commutator factor
    is_solvable(G) && !(is_cyclic(center(G)[1]) &&
                        is_cyclic(maximal_abelian_quotient(G)[1])) && continue

    # omit groups with too many classes
    t = character_table(G)
    m = GAP.Globals.RationalizedMat(GAP.Globals.Irr(GapObj(t)))
    length(m) > 40 && continue

    res = s_characters(t, rational = true, ppow_nonzero = true)
    length(res) != 0 && error("found solvable example ", (n, i))
  end
end